## Definicion de Frameworks

In [1]:
import os
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb

from lightgbm import LGBMRegressor
import lightgbm as lgb
import optuna
import numpy as np

import optuna

## Carga de los datasets de entrenamiento y prueba (TRAIN-TEST)

Descripción de los datasets derivados:
  - train_img.parquet / test_img.parquet:
      Contienen features numéricas extraídas de las imágenes de la competencia (por ejemplo, embeddings o descriptores visuales), generadas en un notebook previo.

  - train_text.parquet / test_text.parquet:
      Contienen features numéricas extraídas del texto asociado a cada mascota (por ejemplo, embeddings o scores de modelos de NLP), también generadas en un notebook previo.

Todos estos archivos fueron exportados a formato Parquet y subidos como dataset de entrada de Kaggle en la ruta BASE definida más abajo.

In [ ]:
BASE = "/kaggle/input/inputs-txt-and-img-2"

train_text = pd.read_parquet(f"{BASE}/train_text.parquet")
train_img  = pd.read_parquet(f"{BASE}/train_img.parquet")

test_text = pd.read_parquet(f"{BASE}/test_text.parquet")
test_img  = pd.read_parquet(f"{BASE}/test_img.parquet")

In [3]:
folder =  "/kaggle/input/petfinder-adoption-prediction"
train = pd.read_csv(f"{folder}/train/train.csv")
test = pd.read_csv(f"{folder}/test/test.csv")

### Uso del split oficial train–test de la competencia y unificación de fuentes

Se respeta la partición train–test provista por la competencia y se alinean todas las fuentes de datos (estructurales, texto e imagen) a nivel de PetID. De este modo, cada mascota queda representada por un vector de features numéricas unificado, sobre el cual se entrenará el modelo de regresión.


In [4]:
# Aseguramos que TODO esté indexado por PetID
if "PetID" in train.columns:
    train = train.set_index("PetID")
if "PetID" in test.columns:
    test = test.set_index("PetID")

for df in [train_text, train_img, test_text, test_img]:
    if "PetID" in df.columns:
        df.set_index("PetID", inplace=True)

# Unificamos: base + texto + imagen
train_full = (
    train
    .join(train_text, how="left")
    .join(train_img, how="left")
)

test_full = (
    test
    .join(test_text, how="left")
    .join(test_img, how="left")
)

print("train_full:", train_full.shape)
print("test_full :", test_full.shape)


train_full: (14993, 1310)
test_full : (3972, 1309)


In [ ]:
target_col = "AdoptionSpeed"

# Por seguridad, si AdoptionSpeed quedó como índice (no debería), lo recuperamos
if target_col not in train_full.columns and target_col in train.columns:
    train_full[target_col] = train[target_col]

# Nos quedamos sólo con columnas numéricas para el modelo
numeric_cols = train_full.select_dtypes(include=[np.number]).columns.tolist()

# Removemos el target de las features
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

X = train_full[numeric_cols]
y = train_full[target_col].astype(int)

X_test = test_full[numeric_cols]

print("X shape     :", X.shape)
print("X_test shape:", X_test.shape)
print("Target dist :")
print(y.value_counts())


X shape     : (14993, 1306)
X_test shape: (3972, 1306)
Target dist :
AdoptionSpeed
4    4197
2    4037
3    3259
1    3090
0     410
Name: count, dtype: int64


## Optimización de hiperparámetros del modelo de regresión (LGBMRegressor)

En esta sección se ajustan los hiperparámetros del modelo de regresión (LightGBM) utilizando Optuna. La variable objetivo `AdoptionSpeed` se trata como un target ordinal entero (0–4), pero el modelo se entrena en un espacio continuo.

In [ ]:
# Se suprimen warnings para evitar que saturen la salida del notebook.
import warnings
warnings.filterwarnings("ignore")

callbacks = [
    lgb.early_stopping(100),
    lgb.log_evaluation(0)   # silencioso total
]

### Esquema de CV estratificada

Dado que `AdoptionSpeed` es una variable categórica ordinal con clases desbalanceadas, se utiliza StratifiedKFold. De esta forma, cada pliegue mantiene aproximadamente la misma distribución de clases que el conjunto completo, lo que mejora la estabilidad de la estimación del desempeño del modelo (QWK) y reduce la varianza entre folds.

In [ ]:

from sklearn.model_selection import StratifiedKFold

N_FOLDS = 5
RANDOM_STATE = 42

kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)


In [ ]:
# Parametro que nos permite optimizar el optuna 
# o utilizar los hiperparametros optimizados anteriormente
ejectuar_optuna = 0

Función objetivo de Optuna para la regresión:
   - Se entrena un LGBMRegressor con un conjunto de hiperparámetros propuesto.
   - Se generan predicciones continuas en validación.
   - Se redondean las predicciones al rango entero 0–4.
   - Se calcula el QWK entre verdad y predicción redondeada.

De esta forma, aunque el modelo se entrena como regresión, la búsqueda de hiperparámetros está alineada con la métrica ordenada de la competencia.


In [ ]:

if ejectuar_optuna == 1: 
    def objective_reg(trial):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 16, 128),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 300, 1500),
    
            # fijos
            "random_state": RANDOM_STATE,
            "n_jobs": -1,
            "verbosity": -1,
        }
    
        oof_pred_reg = np.zeros(len(X), dtype=float)
        fold_scores = []
    
        for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y), 1):
            X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
    
            model = LGBMRegressor(**params)
    
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_val, y_val)],
                eval_metric="rmse",
                callbacks=[
                    lgb.early_stopping(100),
                    lgb.log_evaluation(0)
                ],
            )
    
            y_val_pred = model.predict(X_val)
            oof_pred_reg[val_idx] = y_val_pred
    
            # redondeo simple 0–4 para evaluar QWK durante Optuna
            y_val_pred_round = np.clip(np.rint(y_val_pred), 0, 4).astype(int)
            score = cohen_kappa_score(y_val, y_val_pred_round, weights="quadratic")
            fold_scores.append(score)
    
        return float(np.mean(fold_scores))
    
    
    def print_best_callback(study, trial):
        print(f"\n[Trial {trial.number}] value: {trial.value:.5f}")
        print(f"Best so far: {study.best_value:.5f}")
        print(f"Best params: {study.best_params}")
    
    
    study_reg = optuna.create_study(direction="maximize", study_name="lgbm_reg_petfinder")
    study_reg.optimize(
        objective_reg,
        n_trials=100,
        show_progress_bar=True,
        callbacks=[print_best_callback]
    )
    
    print("\n=== OPTUNA REGRESIÓN TERMINADO ===")
    print("Best QWK (redondeando):", study_reg.best_value)
    print("Best params (reg):")
    for k, v in study_reg.best_params.items():
        print(f"  {k}: {v}")
    
    best_params_reg = study_reg.best_params.copy()
    best_params_reg.update({
        "random_state": RANDOM_STATE,
        "n_jobs": -1,
        "verbosity": -1,
    })
else:
    best_params_reg = {
    "learning_rate": 0.017016522979106517,
    "num_leaves": 76,
    "max_depth": 8,
    "min_child_samples": 95,
    "subsample": 0.8512229857507079,
    "colsample_bytree": 0.8186904774386399,
    "reg_alpha": 0.6828936792548929,
    "reg_lambda": 2.2380795158933614e-07,
    "n_estimators": 1328,

    # Ajustes adicionales necesarios
    "random_state": RANDOM_STATE,
    "n_jobs": -1,
    "verbosity": -1,
}

## Entrenamiento final del modelo de regresión con validación cruzada

Se entrena un LGBMRegressor con los mejores hiperparámetros obtenidos.

Para cada fold:
   - Se ajusta el modelo sobre el conjunto de entrenamiento del fold.
   - Se generan predicciones continuas out-of-fold (OOF) para el conjunto de
     validación, que luego se usarán para optimizar los puntos de corte.
   - Se promedian las predicciones sobre el conjunto de test a lo largo de
     los folds.

Esta etapa entrena un modelo puramente de regresión sobre `AdoptionSpeed` tratado como variable numérica ordinal. La discretización en clases 0–4 se realiza recién en la siguiente sección, mediante cortes óptimos diseñados
para maximizar el QWK.

In [10]:

oof_pred_reg = np.zeros(len(X), dtype=float)
test_pred_reg = np.zeros(len(X_test), dtype=float)

for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y), 1):
    print(f"Fold {fold}/{N_FOLDS}")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    model = LGBMRegressor(**best_params_reg)

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
        callbacks=[
            lgb.early_stopping(100),
            lgb.log_evaluation(0)
        ],
    )

    # OOF continuos
    y_val_pred = model.predict(X_val)
    oof_pred_reg[val_idx] = y_val_pred

    # Predicciones sobre test (promedio de folds)
    test_pred_reg += model.predict(X_test) / N_FOLDS

# QWK con redondeo simple (antes de optimizar cortes, solo para ver)
oof_round = np.clip(np.rint(oof_pred_reg), 0, 4).astype(int)
qwk_round = cohen_kappa_score(y, oof_round, weights="quadratic")
print(f"\nQWK OOF redondeando sin cortes óptimos: {qwk_round:.5f}")


Fold 1/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[493]	valid_0's rmse: 1.06208	valid_0's l2: 1.12801
Fold 2/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[636]	valid_0's rmse: 1.04984	valid_0's l2: 1.10217
Fold 3/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[749]	valid_0's rmse: 1.06399	valid_0's l2: 1.13208
Fold 4/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[602]	valid_0's rmse: 1.06758	valid_0's l2: 1.13973
Fold 5/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[820]	valid_0's rmse: 1.05114	valid_0's l2: 1.10489

QWK OOF redondeando sin cortes óptimos: 0.30005


## Discretización del output continuo del modelo de regresión

El modelo LGBMRegressor produce predicciones continuas que pueden interpretarse como una "propensión" o puntuación latente asociada a la velocidad de adopción.

Dado que la métrica oficial de la competencia es el Quadratic Weighted Kappa (QWK) sobre las clases discretas 0–4, se optimizan puntos de corte (thresholds) sobre estas predicciones continuas.

Usamos Optuna para buscar el conjunto de 4 cortes que maximiza el QWK entre:
   - Las clases verdaderas (0–4), y
   - Las clases predichas al discretizar la salida continua del modelo.

Esta estrategia permite:
  - Tratar `AdoptionSpeed` como una variable ordinal continua en la fase de entrenamiento (regresión), aprovechando bien la estructura del problema.
  - Ajustar después los cortes de forma data-driven para alinear el modelo con la métrica ordinal QWK.

In [11]:
def optimize_cuts(oof_pred_reg, y_true):
    def objective_cuts(trial):
        # 4 cortes dentro del rango [0, 4]
        t1 = trial.suggest_float("t1", 0.0, 4.0)
        t2 = trial.suggest_float("t2", 0.0, 4.0)
        t3 = trial.suggest_float("t3", 0.0, 4.0)
        t4 = trial.suggest_float("t4", 0.0, 4.0)

        cuts = sorted([t1, t2, t3, t4])

        # aplicar cortes
        y_pred = np.digitize(oof_pred_reg, cuts)
        y_pred = np.clip(y_pred, 0, 4)

        return cohen_kappa_score(y_true, y_pred, weights="quadratic")

    study_cuts = optuna.create_study(direction="maximize", study_name="cuts_qwk")
    study_cuts.optimize(objective_cuts, n_trials=200, show_progress_bar=True)

    return study_cuts.best_params, study_cuts.best_value


best_cuts_params, best_cuts_qwk = optimize_cuts(oof_pred_reg, y)
print("\n=== CORTES ÓPTIMOS ENCONTRADOS ===")
print("Best QWK con cortes:", best_cuts_qwk)
print("Cortes (sin ordenar):", best_cuts_params)

# ordenamos los cortes en una lista
cuts = sorted([best_cuts_params["t1"],
               best_cuts_params["t2"],
               best_cuts_params["t3"],
               best_cuts_params["t4"]])

print("Cortes ordenados:", cuts)


[I 2025-12-04 13:28:30,240] A new study created in memory with name: cuts_qwk


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2025-12-04 13:28:30,257] Trial 0 finished with value: 0.16898878580835652 and parameters: {'t1': 3.4262975817717787, 't2': 2.8102348126834524, 't3': 3.402541288584083, 't4': 0.531610735691205}. Best is trial 0 with value: 0.16898878580835652.
[I 2025-12-04 13:28:30,262] Trial 1 finished with value: 0.07362883606253345 and parameters: {'t1': 0.3472000682760772, 't2': 0.5543379887216267, 't3': 2.0388789196611548, 't4': 0.9584682014562023}. Best is trial 0 with value: 0.16898878580835652.
[I 2025-12-04 13:28:30,269] Trial 2 finished with value: 0.2926172454544128 and parameters: {'t1': 1.2698389031507, 't2': 3.8205821830927174, 't3': 2.4634059931613588, 't4': 1.644460825030856}. Best is trial 2 with value: 0.2926172454544128.
[I 2025-12-04 13:28:30,276] Trial 3 finished with value: 0.16551249157557968 and parameters: {'t1': 2.858995157996228, 't2': 0.6738007492866926, 't3': 0.7110038456855445, 't4': 0.5975215569724286}. Best is trial 2 with value: 0.2926172454544128.
[I 2025-12-04 13:2

## Generación del archivo de submission para Kaggle

A partir de las predicciones continuas sobre el set de test y de los cortes óptimos obtenidos, se discretizan las probabilidades en las clases 0–4 de `AdoptionSpeed`. Luego se arma el archivo `submission.csv` con el formato exigido por la competencia (columnas: PetID, AdoptionSpeed).

In [12]:
# aplicar cortes a OOF
oof_pred_final = np.digitize(oof_pred_reg, cuts)
oof_pred_final = np.clip(oof_pred_final, 0, 4)

final_qwk = cohen_kappa_score(y, oof_pred_final, weights="quadratic")
print(f"\nQWK OOF final con cortes óptimos: {final_qwk:.5f}")

# aplicar cortes a test
test_pred_final = np.digitize(test_pred_reg, cuts)
test_pred_final = np.clip(test_pred_final, 0, 4).astype(int)

submission_reg = pd.DataFrame({
    "PetID": test_full.index,
    "AdoptionSpeed": test_pred_final,
})

submission_reg.to_csv("submission.csv", index=False)
print("\nArchivo 'submission.csv' generado.")
print(submission_reg.head())



QWK OOF final con cortes óptimos: 0.41276

Archivo 'submission.csv' generado.
       PetID  AdoptionSpeed
0  e2dfc2935              3
1  f153b465f              2
2  3c90f3f54              1
3  e02abc8a3              4
4  09f0df7d1              3
